In [1]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
import gzip

# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'

# data info
# data path
path_csi =  'J:\\Data\\Wi-Fi_processed\\'
# data info
df_info = pd.read_csv('data_subc_sig.csv')

# parameters
max_id = np.max(df_info['id_person'])
max_len = int(np.max(df_info['len']))
learning_rate = 0.1
training_epochs = 600
batch_size = 300

C:\Users\herok\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# make data generator
def gen_csi(df_info,id_num,len_num):

    for file in set(df_info.id.values):
        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)

        # normalize through subc axis
        abs_sub = np.mean(np.abs(data1),axis=(0,2,3))
        data1_norm = data1/abs_sub[np.newaxis,:,np.newaxis,np.newaxis]

        data1_abs = np.abs(data1_norm)
        data1_ph = np.angle(data1_norm)
        
        # differentiation
        data1_diff = np.diff(data1_abs,axis=0)
        # zero pad
        pad_len = len_num - data1_diff.shape[0]
        data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)
        # reshape
        data1_resh = data1_pad.reshape(30,max_len,6)
        # Label
        data1_lab = df_info[df_info.id==file]['id_person'].values[0]
        # One hot
        data1_one = np.eye(id_num+1)[data1_lab]
        data1_arrays = [data1_one for one in range(30)]
        data1_stack = np.stack(data1_arrays, axis=0)
        
        for i in range(len(data1_resh)):
            yield(data1_resh[i],data1_stack[i])

In [3]:
generator =gen_csi(df_info,max_id,max_len)
data_s,data_l = next(generator)
data_s.shape, data_l.shape

((14960, 6), (108,))

In [4]:
# Load Dataset
from sklearn.model_selection import train_test_split
tr_idx,te_idx = train_test_split(df_info.index,test_size=0.2,random_state=10)
#id_num = len(np.unique(df_info.id_person))
df_train = df_info.loc[tr_idx]
df_test = df_info.loc[te_idx]

gen = lambda: (r for r in gen_csi(df_info,max_id,max_len))
train_dataset = tf.data.Dataset().from_generator(gen, (tf.float32,tf.int32)).shuffle(1000).repeat().batch(batch_size)

# create general iterator
iterator = tf.data.Iterator.from_structure(train_dataset.output_types,train_dataset.output_shapes)
next_element = iterator.get_next()

# make datasets that we can initialize separately, but using the same structure via the common iterator
training_init_op = iterator.make_initializer(train_dataset)

In [ ]:
def CNNmodel(in_data):
    input1d = tf.reshape(in_data, [-1,max_len,6])
    conv1 = tf.layers.conv1d(
        inputs=input1d, 
        filters=100, 
        kernel_size=3, 
        strides=1,
        #padding="same", 
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling1d(
        inputs=conv1, 
        pool_size=10, 
        strides=10)
        #padding="same")
    conv2 = tf.layers.conv1d(
        inputs=pool1, 
        filters=250, 
        kernel_size=3, 
        strides=1,
        #padding="same", 
        activation=tf.nn.relu)
    pool2 = tf.layers.max_pooling1d(
        inputs=conv2, 
        pool_size=10, 
        strides=10)
        #padding="same")
    pool_flat = tf.layers.flatten(pool2)
    fc = tf.layers.dense(
        inputs= pool_flat, units=500, activation=tf.nn.relu)
    output = tf.layers.dense(inputs=fc, units=max_id+1)    
    return output

In [6]:
# define cost/loss & optimizer
logits = CNNmodel(next_element[0])
labels = next_element[1]
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=labels))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# accuracy
pred = tf.argmax(logits,1)
equal = tf.equal(pred,tf.argmax(labels,1))
accuracy = tf.reduce_mean(tf.cast(equal,tf.float32))

#init
init_op = tf.global_variables_initializer()
training_init_op = iterator.make_initializer(train_dataset)

In [7]:
# run the training
epochs = training_epochs
results = pd.DataFrame(columns=['loss','accuracy'])
with tf.Session() as sess:
    sess.run(init_op)
    sess.run(training_init_op)
    for i in range(epochs):
        log,lab,pr = sess.run([logits,labels,pred])
        l, _, acc = sess.run([loss, optimizer, accuracy])
        
        print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i+1, l, acc * 100))
        '''
        # now setup the validation run
        test_iters = 1
        # re-initialize the iterator, but this time with validation data
        sess.run(test_init_op)
        test_l, test_acc = sess.run([loss, accuracy])
        print("Epoch: {}, test loss: {:.3f}, test accuracy: {:.2f}%".format(i+1, test_l, test_acc * 100))
        '''
        results.loc[i] = [l,acc]

Epoch: 1, loss: 4.681, training accuracy: 0.00%
Epoch: 2, loss: 100224696.000, training accuracy: 5.33%
Epoch: 3, loss: 267875.188, training accuracy: 0.00%
Epoch: 4, loss: 347.107, training accuracy: 4.00%
Epoch: 5, loss: 178620.078, training accuracy: 0.00%
Epoch: 6, loss: 31587.486, training accuracy: 0.00%
Epoch: 7, loss: 249.996, training accuracy: 1.00%
Epoch: 8, loss: 263.833, training accuracy: 2.00%
Epoch: 9, loss: 138.905, training accuracy: 5.33%
Epoch: 10, loss: 67.350, training accuracy: 0.67%
Epoch: 11, loss: 30.863, training accuracy: 0.00%
Epoch: 12, loss: 36.220, training accuracy: 4.00%
Epoch: 13, loss: 813.858, training accuracy: 0.00%
Epoch: 14, loss: 19.365, training accuracy: 0.00%
Epoch: 15, loss: 7.503, training accuracy: 1.33%
Epoch: 16, loss: 8.147, training accuracy: 1.67%
Epoch: 17, loss: 11.092, training accuracy: 0.00%
Epoch: 18, loss: 9.936, training accuracy: 0.00%
Epoch: 19, loss: 9.492, training accuracy: 0.00%
Epoch: 20, loss: 9.138, training accuracy

Epoch: 174, loss: 4.767, training accuracy: 2.33%
Epoch: 175, loss: 5.607, training accuracy: 1.33%
Epoch: 176, loss: 5.463, training accuracy: 0.00%
Epoch: 177, loss: 5.377, training accuracy: 0.00%
Epoch: 178, loss: 4.713, training accuracy: 0.00%
Epoch: 179, loss: 5.558, training accuracy: 4.67%
Epoch: 180, loss: 5.246, training accuracy: 2.00%
Epoch: 181, loss: 4.660, training accuracy: 4.00%
Epoch: 182, loss: 5.645, training accuracy: 1.00%
Epoch: 183, loss: 5.157, training accuracy: 2.33%
Epoch: 184, loss: 5.477, training accuracy: 2.67%
Epoch: 185, loss: 5.228, training accuracy: 1.67%
Epoch: 186, loss: 5.382, training accuracy: 0.00%
Epoch: 187, loss: 4.762, training accuracy: 1.33%
Epoch: 188, loss: 4.764, training accuracy: 1.67%
Epoch: 189, loss: 5.112, training accuracy: 0.33%
Epoch: 190, loss: 4.686, training accuracy: 0.00%
Epoch: 191, loss: 5.054, training accuracy: 0.67%
Epoch: 192, loss: 4.717, training accuracy: 0.67%
Epoch: 193, loss: 4.990, training accuracy: 2.00%


Epoch: 338, loss: 4.541, training accuracy: 1.33%
Epoch: 339, loss: 4.524, training accuracy: 3.33%
Epoch: 340, loss: 4.856, training accuracy: 0.33%
Epoch: 341, loss: 4.649, training accuracy: 2.67%
Epoch: 342, loss: 4.705, training accuracy: 2.33%
Epoch: 343, loss: 4.850, training accuracy: 1.00%
Epoch: 344, loss: 4.604, training accuracy: 1.00%
Epoch: 345, loss: 4.814, training accuracy: 0.67%
Epoch: 346, loss: 4.774, training accuracy: 0.00%
Epoch: 347, loss: 4.856, training accuracy: 0.33%
Epoch: 348, loss: 4.705, training accuracy: 0.67%
Epoch: 349, loss: 4.588, training accuracy: 1.33%
Epoch: 350, loss: 4.604, training accuracy: 1.33%
Epoch: 351, loss: 4.496, training accuracy: 0.33%
Epoch: 352, loss: 4.496, training accuracy: 2.00%
Epoch: 353, loss: 4.456, training accuracy: 1.00%
Epoch: 354, loss: 4.690, training accuracy: 0.67%
Epoch: 355, loss: 4.584, training accuracy: 0.00%
Epoch: 356, loss: 4.628, training accuracy: 2.67%
Epoch: 357, loss: 4.611, training accuracy: 0.00%


Epoch: 502, loss: 4.893, training accuracy: 0.00%
Epoch: 503, loss: 4.794, training accuracy: 1.00%
Epoch: 504, loss: 4.690, training accuracy: 5.00%
Epoch: 505, loss: 4.875, training accuracy: 0.67%
Epoch: 506, loss: 4.588, training accuracy: 0.00%
Epoch: 507, loss: 4.984, training accuracy: 0.00%
Epoch: 508, loss: 4.751, training accuracy: 6.00%
Epoch: 509, loss: 5.065, training accuracy: 2.00%
Epoch: 510, loss: 4.969, training accuracy: 1.00%
Epoch: 511, loss: 4.900, training accuracy: 1.00%
Epoch: 512, loss: 4.712, training accuracy: 4.67%
Epoch: 513, loss: 5.046, training accuracy: 0.33%
Epoch: 514, loss: 5.204, training accuracy: 0.00%
Epoch: 515, loss: 5.080, training accuracy: 0.33%
Epoch: 516, loss: 5.207, training accuracy: 4.00%
Epoch: 517, loss: 4.861, training accuracy: 2.00%
Epoch: 518, loss: 5.230, training accuracy: 0.00%
Epoch: 519, loss: 4.577, training accuracy: 5.33%
Epoch: 520, loss: 4.742, training accuracy: 6.33%
Epoch: 521, loss: 4.743, training accuracy: 4.00%


In [5]:
def CNNmodel(in_data):
    input1d = tf.reshape(in_data, [-1,max_len,6])
    conv11 = tf.layers.conv1d(
        inputs=input1d, 
        filters=32, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv12 = tf.layers.conv1d(
        inputs=conv11, 
        filters=32, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    pool1 = tf.layers.max_pooling1d(
        inputs=conv12, 
        pool_size=10, 
        strides=10)
    conv21 = tf.layers.conv1d(
        inputs=pool1, 
        filters=64, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv211 = tf.layers.conv1d(
        inputs=conv21, 
        filters=32, 
        kernel_size=1, 
        strides=1,
        padding="same", 
        activation=None)
    conv22 = tf.layers.conv1d(
        inputs=conv211, 
        filters=64, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    
    pool2 = tf.layers.max_pooling1d(
        inputs=conv22, 
        pool_size=10,
        strides=10)
    
    conv31 = tf.layers.conv1d(
        inputs=pool2, 
        filters=128, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv311 = tf.layers.conv1d(
        inputs=conv31, 
        filters=64, 
        kernel_size=1, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv32 = tf.layers.conv1d(
        inputs=conv311, 
        filters=128, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv321 = tf.layers.conv1d(
        inputs=conv32, 
        filters=64, 
        kernel_size=1, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    conv33 = tf.layers.conv1d(
        inputs=conv321, 
        filters=128, 
        kernel_size=3, 
        strides=1,
        padding="same", 
        activation=tf.nn.relu)
    
    pool3 = tf.layers.max_pooling1d(
        inputs=conv33, 
        pool_size=10, 
        strides=10)
    
    pool_flat = tf.layers.flatten(pool3)
    fc = tf.layers.dense(
        inputs= pool_flat, units=500)#, activation=tf.nn.relu)
    #fc_drop = tf.nn.dropout(fc, keep_prob) 
    output = tf.layers.dense(inputs=fc, units=max_id+1) 
    return output

In [ ]:
pr